<a href="https://colab.research.google.com/github/mohzary/python-deep-learning-f19/blob/master/M2ICP5/M2ICP5_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## In class programming:
1. Save the model and use the saved model to predict on new text data (ex, “A lot of good things are happening. We are respected again throughout the world, and that's a great thing.@realDonaldTrump”).

2. Apply GridSearchCV on the source code provided in the class

In [1]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
import io
from google.colab import files
from sklearn.preprocessing import LabelEncoder

Using TensorFlow backend.


In [2]:
# To upload data file from local folder to colab
data_file = files.upload()


Saving Sentiment.csv to Sentiment.csv


In [0]:
data = pd.read_csv(io.StringIO(data_file['Sentiment.csv'].decode('utf-8')))


In [4]:
data.head(5)

,id,candidate,candidate_confidence,relevant_yn,relevant_yn_confidence,sentiment,sentiment_confidence,subject_matter,subject_matter_confidence,candidate_gold,name,relevant_yn_gold,retweet_count,sentiment_gold,subject_matter_gold,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,1,No candidate mentioned,1.0,yes,1.0,Neutral,0.6578,None of the above,1.0000,NaN,I_Am_Kenzi,NaN,5,NaN,NaN,RT @NancyLeeGrahn: How did everyone feel about...,NaN,2015-08-07 09:54:46 -0700,629697200650592256,NaN,Quito
1,2,Scott Walker,1.0,yes,1.0,Positive,0.6333,None of the above,1.0000,NaN,PeacefulQuest,NaN,26,NaN,NaN,RT @ScottWalker: Didn't catch the full #GOPdeb...,NaN,2015-08-07 09:54:46 -0700,629697199560069120,NaN,NaN
2,3,No candidate mentioned,1.0,yes,1.0,Neutral,0.6629,None of the above,0.6629,NaN,PussssyCroook,NaN,27,NaN,NaN,RT @TJMShow: No mention of Tamir Rice and the ...,NaN,2015-08-07 09:54:46 -0700,629697199312482304,NaN,NaN
3,4,No candidate mentioned,1.0,yes,1.0,Positive,1.0000,None of the above,0.7039,NaN,MattFromTexas31,NaN,138,NaN,NaN,RT @RobGeorge: That Carly Fiorina is trending ...,NaN,2015-08-07 09:54:45 -0700,629697197118861312,Texas,Central Time (US & Canada)
4,5,Donald Trump,1.0,yes,1.0,Positive,0.7045,None of the above,1.0000,NaN,sharonDay5,NaN,156,NaN,NaN,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,NaN,2015-08-07 09:54:45 -0700,629697196967903232,NaN,Arizona


In [0]:
# Keeping only the neccessary columns
data = data[['text','sentiment']]

# to return the lowercased string from the given string. It converts all uppercase characters to lowercase. If no uppercase characters exist, it returns the original string.
data['text'] = data['text'].apply(lambda x: x.lower())

data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ')


In [6]:
data.head(5)

,text,sentiment
0,nancyleegrahn how did everyone feel about th...,Neutral
1,scottwalker didnt catch the full gopdebate l...,Positive
2,tjmshow no mention of tamir rice and the gop...,Neutral
3,robgeorge that carly fiorina is trending ho...,Positive
4,danscavino gopdebate w realdonaldtrump deliv...,Positive


In [0]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)

In [0]:
X = pad_sequences(X)

embed_dim = 128
lstm_out = 196

In [0]:
def createmodel():
    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(3,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    return model
#print(model.summary())

In [0]:
labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['sentiment'])
y = to_categorical(integer_encoded)

In [0]:
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)


In [18]:
from keras.callbacks import TensorBoard
from keras.wrappers.scikit_learn import KerasClassifier

tbCallBack= TensorBoard(log_dir='./GraphICP5-2', histogram_freq=0,write_graph=True, write_images=True) 
batch_size = 32
model = KerasClassifier(build_fn=createmodel,verbose=0)
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs = 10, batch_size=batch_size, verbose = 2, callbacks=[tbCallBack])

Train on 9293 samples, validate on 4578 samples
Epoch 1/10
 - 18s - loss: 0.8251 - acc: 0.6451 - val_loss: 0.7751 - val_acc: 0.6544
Epoch 2/10
 - 18s - loss: 0.6837 - acc: 0.7056 - val_loss: 0.7472 - val_acc: 0.6804
Epoch 3/10
 - 18s - loss: 0.6258 - acc: 0.7372 - val_loss: 0.7741 - val_acc: 0.6689
Epoch 4/10
 - 17s - loss: 0.5792 - acc: 0.7619 - val_loss: 0.7736 - val_acc: 0.6804
Epoch 5/10
 - 17s - loss: 0.5412 - acc: 0.7796 - val_loss: 0.8168 - val_acc: 0.6621
Epoch 6/10
 - 17s - loss: 0.5002 - acc: 0.7961 - val_loss: 0.8495 - val_acc: 0.6649
Epoch 7/10
 - 17s - loss: 0.4595 - acc: 0.8093 - val_loss: 0.9112 - val_acc: 0.6632
Epoch 8/10
 - 17s - loss: 0.4255 - acc: 0.8273 - val_loss: 0.9865 - val_acc: 0.6599
Epoch 9/10
 - 17s - loss: 0.4004 - acc: 0.8391 - val_loss: 1.0862 - val_acc: 0.6608
Epoch 10/10
 - 18s - loss: 0.3798 - acc: 0.8500 - val_loss: 1.1090 - val_acc: 0.6492


In [22]:
from sklearn.model_selection import GridSearchCV
batch_size= [10, 20]
epochs = [1, 2]
param_grid= {'batch_size':batch_size, 'epochs':epochs}

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result= grid.fit(X_train, Y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


In [21]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.669321 using {'batch_size': 10, 'epochs': 1}
